In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


In [2]:
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import re

In [3]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
driver.get('https://www.cars24.com/buy-used-car/')

In [4]:
def load_till_last():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.60);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        sleep(3)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    driver.execute_script("window.scrollTo(0, 100);")
    return None

In [5]:
[i.text for i in driver.find_elements(By.CLASS_NAME, '_2kfVy')]

['2013 Mahindra Bolero\nSLX BS IVManual\n83,078 KM\n2ND OWNER\nDIESEL\nHR-03\n₹8,798/month\nZero Down Payment\n₹4,50,000\nParas Downtown Square, Chandigarh',
 '₹54,000\noff\n2018 Maruti IGNIS\nSIGMA 1.2Manual\n35,874 KM\n2ND OWNER\nPETROL\nDL-12\n₹7,761/month\nZero Down Payment\n₹3,97,000\n₹4,51,000\nM3M Urbana, Golf Course Extension, Gurugram',
 '2011 Maruti Ritz\nVXIManual\n64,449 KM\n2ND OWNER\nPETROL\nMH-01\n₹3,675/month\nZero Down Payment\n₹1,88,000\nKharbi, Nagpur',
 '₹5,000\noff\n2010 Maruti Alto\nLXIManual\n49,984 KM\n1ST OWNER\nPETROL\nUP-14\n₹2,385/month\nZero Down Payment\n₹1,22,000\n₹1,27,000\nSpectrum Metro, Sector 75, Noida',
 '2011 Hyundai Verna\nFLUIDIC 1.6 CRDI SXManual\n95,877 KM\n3RD OWNER\nDIESEL\nMH-12\n₹7,957/month\nZero Down Payment\n₹4,07,000\nTathawade, Pune',
 '2013 Hyundai Verna\nFLUIDIC 1.6 CRDI SXManual\n1,22,321 KM\n2ND OWNER\nDIESEL\nMH-49\n₹9,091/month\nZero Down Payment\n₹4,65,000\nKharbi, Nagpur',
 '₹26,000\noff\n2011 Honda City\nS MT PETROLManual\n89,

In [6]:
XPath_to_model_list = '/html/body/div[1]/div[1]/div[2]/div/div/div[1]/div/div[1]/div[3]/ul'

In [7]:
def SplitEngineDetail(html_list):
    transmission = html_list[2].text
    engine_type = html_list[0].text
    return engine_type, transmission

In [8]:
def SplitCarDetails(html_list):
    kms_driven = html_list[0].text
    owner_type = html_list[1].text
    fuel_type = html_list[2].text
    passing = html_list[3].text
    return kms_driven, owner_type, fuel_type, passing

In [9]:
def SplitTitle(text):
    title_list = text.split(' ')
    year = title_list[0]
    model = ' '.join(title_list[2:])
    cmpny = title_list[1]
    return year, cmpny, model

In [10]:
for car_element in driver.find_elements(By.CLASS_NAME, '_2kfVy')[15:]:
    html_cntnt = car_element.get_attribute('outerHTML')
    soup = BeautifulSoup(html_cntnt, 'html.parser')
    print(SplitTitle( soup.find('h2', {'class':'_2lmIw'}).text))
    print(SplitEngineDetail(soup.find('ul',{'class':'_1hOnS'}).findChildren()))
    print(SplitCarDetails(soup.find('ul',{'class':'_13yb6'}).findChildren()))
    print(soup.find('div',{'class':'_18ToE'}).findChildren()[0].text[1:])
    print(soup.find('span',{'class':'_3DYbK'}).text.split(',')[-1])
    break

('2011', 'Toyota', 'Etios')
('G', 'Manual')
('73,333 km', '2nd Owner', 'Petrol', 'DL-3C')
1,77,000
 New Delhi


In [11]:
def create_dictionary():
    return {
        'Year':[],
        'Company':[],
        'Model':[],
        'Transmission':[],
        'Engine_Type':[],
        'Kms_Driven':[],
        'ownership':[],
        'Fuel_Type':[],
        'Passing':[],
        'Area':[],
        'Price':[],
    }

In [12]:
def get_car_details(dictionary, soup):
    year, cmpny, model = SplitTitle(soup.find('h2', {'class':'_2lmIw'}).text)
    engine_type, transmission = SplitEngineDetail(soup.find('ul',{'class':'_1hOnS'}).findChildren())
    kms_driven, owner_type, fuel_type, passing = SplitCarDetails(soup.find('ul',{'class':'_13yb6'}).findChildren())
    price = soup.find('div',{'class':'_18ToE'}).findChildren()[0].text[1:]
    area = soup.find('span',{'class':'_3DYbK'}).text.split(',')[-1]
    dictionary['Year'].append(year)
    dictionary['Company'].append(cmpny)
    dictionary['Model'].append(model)
    dictionary['Transmission'].append(transmission)
    dictionary['Engine_Type'].append(engine_type)
    dictionary['Kms_Driven'].append(kms_driven)
    dictionary['ownership'].append(owner_type)
    dictionary['Fuel_Type'].append(fuel_type)
    dictionary['Passing'].append(passing)
    dictionary['Area'].append(area.replace('\n', ''))
    dictionary['Price'].append(price)
    
    return dictionary       
    


In [13]:
def get_current__page_data():
    dictionary = create_dictionary()
    for car_element in driver.find_elements(By.CLASS_NAME, '_2kfVy'):
        html_cntnt = car_element.get_attribute('outerHTML')
        soup = BeautifulSoup(html_cntnt, 'html.parser')
        dictionary = get_car_details(dictionary, soup)

    return pd.DataFrame(dictionary)

In [14]:
def traverse_through_cmpny(car_cmpny_list):
    num = 14
    df = pd.DataFrame()
    # df = pd.read_csv('Cars_24.csv')
    for cmpny in car_cmpny_list[num-2:]:
        # select cmpny 
        sleep(2)
        driver.find_element(By.XPATH,f"{XPath_to_model_list}[{num}]/li/div/div/div/div").click()
        load_till_last()
        df = pd.concat([df, get_current__page_data()])
        
        # deselect cmpny
        sleep(2)
        driver.find_element(By.XPATH,f"{XPath_to_model_list}[{num}]/li/div/div/div/div").click()
        num+=1
        df.to_csv('Cars_24.csv', index=None)
        print(cmpny)
        

In [15]:
def get_data():
    car_cmpny_list = [i.text for i in driver.find_elements(By.XPATH, XPath_to_model_list)[1:] if '(0)' not in i.text]
    return traverse_through_cmpny(car_cmpny_list)

In [16]:
get_data()

Nissan (55)
Jeep (6)
MG (4)
Bmw (1)
